# Tutorial

In this tutorial we fairly compare a number of ensemble methods using EI's built in nested cross-validation implementation, and show how predictions can be made with the selected final model. We then show how we can intepret the model by calculating feature rankings.

### Performance analysis and selection of ensemble methods

First of all let's import some `sklearn` models, `EnsembleIntegration` and some additional ensemble methods:

In [19]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.ei import EnsembleIntegration
from eipy.additional_ensembles import MeanAggregation, CES

Next make some dummy "multi-modal" data from the breast cancer dataset:

In [60]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
feature_names = data.feature_names
X = data.data
y = np.abs(data.target - 1)  # make "malignancy" the positive class rather than "benign"

X_1 = X[:, 0:10]
X_2 = X[:, 10:]

X_1_train, X_1_test, y_train, y_test = train_test_split(X_1, y, test_size=0.2, random_state=3, stratify=y)
X_2_train, X_2_test, _, _ = train_test_split(X_2, y, test_size=0.2, random_state=3, stratify=y)

Create dictionaries containing data modalities:

In [61]:
data_train = {
                "Modality_1": X_1_train,
                "Modality_2": X_2_train
                }

data_test = {
                "Modality_1": X_1_test,
                "Modality_2": X_2_test
                }

Define base predictors:

In [62]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(), 
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True),
}

Initialise Ensemble Integration:

In [63]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy="undersampling",
                        sampling_aggregation="mean",
                        n_jobs=-1,
                        random_state=38,
                        project_name="breast_cancer",
                        model_building=True,
                        )

Train base predictors on each modality. Remember to include the unique modality name.

In [64]:
for name, modality in data_train.items():
    EI.train_base(modality, y_train, modality=name)

Training base predictors on Modality_1...

... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on Modality_2...

... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


We can check the cross validated performance of each base predictor on each modality with `base_summary`:

In [65]:
EI.base_summary['metrics']

modality        Modality_1                                                    \
base predictor        ADAB        DT        GB       KNN        LR       MLP   
fmax (minority)   0.918129  0.890173  0.919075  0.836013  0.882022  0.825397   
f (majority)      0.950704  0.932624  0.950355  0.914858  0.924188  0.907563   
AUC               0.977802  0.914345  0.977564  0.917214  0.970320  0.924272   
max MCC           0.856092  0.823152  0.872648  0.759582  0.811426  0.745580   

modality                                                Modality_2            \
base predictor         NB        RF       SVM       XGB       ADAB        DT   
fmax (minority)  0.896359  0.912181  0.844156  0.928994   0.952663  0.902857   
f (majority)     0.933092  0.944345  0.920266  0.958042   0.972028  0.939286   
AUC              0.972621  0.974592  0.935459  0.978803   0.984128  0.926109   
max MCC          0.830594  0.858614  0.775230  0.887078   0.919982  0.843132   

modality                                                                     \
base predictor         GB       KNN        LR       MLP        NB        RF   
fmax (minority)  0.935385  0.896755  0.936047  0.895522  0.935294  0.941896   
f (majority)     0.964103  0.938704  0.961131  0.939130  0.961404  0.967410   
AUC              0.984004  0.958524  0.988524  0.963715  0.987255  0.987513   
max MCC          0.897333  0.835469  0.897343  0.841350  0.892473  0.911979   

modality                             
base predictor        SVM       XGB  
fmax (minority)  0.882540  0.943620  
f (majority)     0.937815  0.966841  
AUC              0.966130  0.984479  
max MCC          0.822439  0.910852

Now let's define some meta models for stacked generalization. We add an "S." prefix to the keys of stacking algorithms.

In [66]:
meta_predictors = {     
                    'Mean' : MeanAggregation(),
                    'CES' : CES(),
                    'S.ADAB': AdaBoostClassifier(),
                    'S.XGB': XGBClassifier(),
                    'S.DT': DecisionTreeClassifier(),
                    "S.RF": RandomForestClassifier(), 
                    'S.GB': GradientBoostingClassifier(),
                    'S.KNN': KNeighborsClassifier(),
                    'S.LR': LogisticRegression(),
                    'S.NB': GaussianNB(),
                    'S.MLP': MLPClassifier(),
                    'S.SVM': SVC(probability=True),
}

Train meta models:

In [67]:
EI.train_meta(meta_predictors=meta_predictors)

Analyzing ensembles: |██████████|100%
Training final meta models: |██████████|100%


Check the meta summary with `meta_summary`:

In [68]:
EI.meta_summary['metrics']

Mean       CES    S.ADAB     S.XGB      S.DT      S.RF  \
fmax (minority)  0.939130  0.947368  0.932153  0.948328  0.923077  0.950920   
f (majority)     0.962832  0.968310  0.959720  0.970740  0.954545  0.972603   
AUC              0.988710  0.990134  0.977141  0.986151  0.937771  0.985738   
max MCC          0.902221  0.915720  0.891884  0.920696  0.877664  0.926137   

                     S.GB     S.KNN      S.LR      S.NB     S.MLP     S.SVM  
fmax (minority)  0.950437  0.943620  0.953488  0.941860  0.956012  0.949853  
f (majority)     0.970018  0.966841  0.971731  0.964664  0.973638  0.970228  
AUC              0.980361  0.976966  0.988380  0.975418  0.983385  0.979711  
max MCC          0.921100  0.910557  0.925388  0.902221  0.929660  0.920091

The MLP stacking algorithm has the best $\text{F}_\text{max}$ performance (the preferred metric for imbalanced datasets) so let's select it as our final model.

### Predictions on unseen data

Since we ran EI with `model_building=True`, we can make predictions. Let's predict the test set and apply the $\text{F}_\text{max}$ threshold calculated during training:

In [70]:
y_pred = EI.predict(X_dict=data_test, meta_model_key='S.MLP')

threshold = EI.meta_summary['thresholds']['S.MLP']['fmax (minority)']

y_pred[y_pred>=threshold] = 1
y_pred[y_pred<threshold] = 0

print(y_pred)

[0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0.]



### Interpreting the final model

We now use `PermutationInterpreter` to interpret the final MLP stacked generalization model. Let's first import `PermutationInterpreter` and our chosen metric, and initialise the interpreter:

In [71]:
from eipy.interpretation import PermutationInterpreter
from eipy.utils import f_minority_score

interpreter = PermutationInterpreter(EI=EI,
                                     metric=f_minority_score,
                                     meta_predictor_keys=['S.MLP'])

Calculate feature importance scores:

In [73]:
interpreter.rank_product_score(X_dict=data_train, y=y_train)

Interpreting ensembles...



Calculating local feature ranks: |██████████|100%
Calculating local model ranks: |██████████|100%

Calculating combined rank product score...
... complete!


We can now inspect the most important features for model prediction:

In [78]:
ranking_dataframe = interpreter.ensemble_feature_ranking['S.MLP']
reordered_feature_names = feature_names[ranking_dataframe.index]
ranking_dataframe['feature'] = reordered_feature_names

ranking_dataframe

modality                  feature       RPS  feature rank  \
3   Modality_1                mean area  0.056500           1.0   
23  Modality_2               worst area  0.108750           2.0   
1   Modality_1             mean texture  0.147250           3.0   
7   Modality_1      mean concave points  0.172500           4.0   
13  Modality_2               area error  0.177750           5.0   
21  Modality_2            worst texture  0.197750           6.0   
27  Modality_2     worst concave points  0.207125           7.0   
6   Modality_1           mean concavity  0.217500           8.0   
2   Modality_1           mean perimeter  0.221000           9.0   
22  Modality_2          worst perimeter  0.241750          10.0   
5   Modality_1         mean compactness  0.247000          11.0   
0   Modality_1              mean radius  0.255750          12.0   
12  Modality_2          perimeter error  0.258125          13.0   
26  Modality_2          worst concavity  0.260000          14.0   
4   Modality_1          mean smoothness  0.261750          15.0   
20  Modality_2             worst radius  0.286375          16.0   
8   Modality_1            mean symmetry  0.314500          17.0   
24  Modality_2         worst smoothness  0.315625          18.0   
9   Modality_1   mean fractal dimension  0.333750          19.0   
10  Modality_2             radius error  0.352500          20.0   
25  Modality_2        worst compactness  0.398000          21.0   
19  Modality_2  fractal dimension error  0.409500          22.0   
15  Modality_2        compactness error  0.418875          23.0   
14  Modality_2         smoothness error  0.421500          24.0   
17  Modality_2     concave points error  0.429875          25.0   
18  Modality_2           symmetry error  0.451750          26.0   
29  Modality_2  worst fractal dimension  0.452750          27.0   
11  Modality_2            texture error  0.455375          28.0   
28  Modality_2           worst symmetry  0.457375          29.0   
16  Modality_2          concavity error  0.471750          30.0   

   ensemble method  
3            S.MLP  
23           S.MLP  
1            S.MLP  
7            S.MLP  
13           S.MLP  
21           S.MLP  
27           S.MLP  
6            S.MLP  
2            S.MLP  
22           S.MLP  
5            S.MLP  
0            S.MLP  
12           S.MLP  
26           S.MLP  
4            S.MLP  
20           S.MLP  
8            S.MLP  
24           S.MLP  
9            S.MLP  
10           S.MLP  
25           S.MLP  
19           S.MLP  
15           S.MLP  
14           S.MLP  
17           S.MLP  
18           S.MLP  
29           S.MLP  
11           S.MLP  
28           S.MLP  
16           S.MLP